# Feed-Forward Neural Network

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib widget

In [71]:
from itertools import product
from pathlib import Path
from tqdm.auto import trange

import numpy as np
import xarray as xr
import xarray.ufuncs as xf
import matplotlib.pyplot as plt
from scipy.interpolate import griddata

from system_identification.ffnn import FeedForwardNeuralNetwork, TrainingParameters
from system_identification.load_assignment_data import load_net_example_ff

In [72]:
nn = FeedForwardNeuralNetwork.new(
    n_inputs=2,
    n_outputs=1,
    n_hidden=12,
    range=[[-1, 1], [-1, 1]],
    training_parameters=TrainingParameters(
        epochs=1000,
        goal=0,
        min_grad=1e-10,
        mu=0.001,
    ),
)

In [73]:
nn

<FeedForwardNeuralNetwork
 n_inputs=2
 n_hidden=12
 n_output=1>

In [74]:
def f(x):
    return 0.7 * np.tanh(0.8 * x[0] - 0.5) + 1

# def f(x):
#     return x[0]**2

def f(x):
    return -0.8 * np.tanh(x[0] * 3) + x[1]**2 + 1


inputs = np.random.uniform(-5, 5, (1000, 2))
reference_outputs = np.array(list(map(f, inputs))).reshape(1000, 1)

In [75]:
nn.back_propagation(inputs, reference_outputs, epochs=1000)

  0%|          | 0/1000 [00:00<?, ?it/s]

In [76]:
# print(nn.input_weights)
# print(nn.output_weights)
# print(nn.bias_weights)

coords = np.linspace(-5, 5, 100)
nn_eval = np.empty((100,))
fn_eval = np.empty((100,))
plt.figure()

for y in np.linspace(-1, 1, 4):
    for xi, x in enumerate(coords):
        fn_eval[xi] = f([x, y])
        nn_eval[xi] = nn.evaluate([x, y])

    plt.plot(coords, fn_eval, "b")
    plt.plot(coords, nn_eval, "r")

# plt.plot(inputs, reference_outputs, ".", markersize=.1)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [81]:
coords = np.linspace(-1, 1, 100)
nn_eval = np.empty((100, 100))
fn_eval = np.empty((100, 100))
for xi, x in enumerate(coords):
    for yi, y in enumerate(coords):
        fn_eval[xi, yi] = f([x, y])
        nn_eval[xi, yi] = nn.evaluate([x, y])

fig = plt.figure()
pos = plt.imshow(fn_eval)
fig.colorbar(pos)

fig = plt.figure()
pos = plt.imshow(nn_eval)
fig.colorbar(pos)

fig = plt.figure()
zi = griddata(inputs[:, ::-1], reference_outputs, (coords[None, :], coords[:, None]), method='cubic')
pos = plt.imshow(zi)
fig.colorbar(pos)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Example

In [144]:
data_dir_path = Path().cwd().parent.parent / "assignment"
example_nn = load_net_example_ff(data_dir_path)
example_nn

In [73]:
x = np.array([[1, 2, 3]]).T
y = np.array([[4, 5, 6, 7]])

x @ y

array([[ 4,  5,  6,  7],
       [ 8, 10, 12, 14],
       [12, 15, 18, 21]])

In [85]:
np.zeros(3)

array([0., 0., 0.])

In [86]:
import sympy as sp

In [120]:
x = sp.Matrix(sp.symbols("x_{00} x_{01} x_{02} x_{10} x_{11} x_{12}")).reshape(2, 3)
y = sp.Matrix(sp.symbols("y_0:3")).T

In [121]:
x

Matrix([
[x_{00}, x_{01}, x_{02}],
[x_{10}, x_{11}, x_{12}]])

In [124]:
y

Matrix([[y_0, y_1, y_2]])

In [129]:
qwerty = sp.zeros(3, 1)
for j, k in product(range(3), range(2)):
    qwerty[j] += x[k, j] * y[j]
qwerty

Matrix([
[x_{00}*y_0 + x_{10}*y_0],
[x_{01}*y_1 + x_{11}*y_1],
[x_{02}*y_2 + x_{12}*y_2]])

ShapeError: Matrix size mismatch: (3, 2) * (1, 3).